# Setting

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install openai==1.55.3 httpx==0.27.2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
openai_api_key = os.getenv('OPENAI_API_KEY')

In [5]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'


# 영화 추천 유튜버를 위한 시나리오 작성 Multi-Agent

## 리포트 버전

In [6]:
## 리포트 버전

# 1️⃣ {movie} 정보 수집 에이전트
movie_researcher = Agent(
    role="{movie} 연구자",
    goal="입력받은 {movie}의 개요, 감독, 출연진, 장르, 제작 배경, 흥행 성적 등의 정보를 조사",
    backstory="당신은 {movie} 전문가로서, 특정 {movie}에 대한 자세한 정보를 수집하는 역할을 맡고 있습니다. "
              "{movie}의 제작 과정, 주요 배우 및 감독의 이력, 촬영 기법 등의 데이터를 신뢰할 수 있는 출처에서 조사합니다. "
              "또한, {movie}가 개봉했을 당시의 평가와 관객 반응도 함께 정리하여 유튜버가 리뷰에 활용할 수 있도록 제공합니다.",
    allow_delegation=False,
    verbose=True
)

# 2️⃣ {movie} 요약 에이전트
movie_summarizer = Agent(
    role="{movie} 요약가",
    goal="{movie}의 주요 내용을 스포일러를 최소화하면서 간결하고 효과적으로 요약",
    backstory="당신은 {movie} 리뷰 콘텐츠를 위한 전문 요약가입니다. "
              "{movie}의 주요 줄거리를 정리하되, 관객의 흥미를 유지할 수 있도록 핵심적인 요소를 강조하고 불필요한 세부사항은 배제합니다. "
              "{movie}의 전반적인 흐름을 전달하면서도, 스토리의 흥미를 유지할 수 있도록 스포일러 여부를 신중하게 고려합니다.",
    allow_delegation=False,
    verbose=True
)

# 3️⃣ {movie} 해석 및 결말 분석 에이전트
movie_analyst = Agent(
    role="{movie} 해석 전문가",
    goal="{movie}의 주제, 상징성, 감독의 연출 의도 등을 분석하고 결말에 대한 해석을 제공",
    backstory="당신은 {movie}의 메시지와 감독의 연출 의도를 심층적으로 분석하는 {movie} 평론가입니다. "
              "{movie}의 장면, 대사, 색채, 카메라 기법 등을 분석하여 {movie}가 전달하려는 의미를 파악합니다. "
              "또한, {movie}의 결말에 대한 다양한 해석을 제시하여 시청자가 더 깊이 이해할 수 있도록 도와줍니다.",
    allow_delegation=False,
    verbose=True
)

# 4️⃣ 비슷한 {movie} 추천 에이전트
movie_recommender = Agent(
    role="{movie} 추천 전문가",
    goal="분석한 {movie}와 유사한 스타일, 장르, 주제를 가진 {movie}를 추천",
    backstory="당신은 {movie} 추천 알고리즘 전문가로서, 특정 {movie}와 유사한 작품들을 분석하고 추천하는 역할을 맡고 있습니다. "
              "장르, 감독, 촬영 기법, 분위기, 스토리 구조 등을 고려하여 유사한 {movie}를 선정합니다. "
              "유튜버가 다양한 추천 리스트를 제공할 수 있도록, 각 추천 {movie}에 대한 간단한 설명과 추천 이유도 함께 작성합니다.",
    allow_delegation=False,
    verbose=True
)

# 5️⃣ {movie} 리뷰 리포트 작성 에이전트
movie_reporter = Agent(
    role="{movie} 리뷰 리포트 작성자",
    goal="{movie}에 대한 종합적인 영화 리뷰 리포트를 작성",
    backstory="당신은 영화 리뷰 콘텐츠를 제작하는 크리에이터를 위한 전문 리포트 작성자입니다. "
              "{movie}에 대한 조사, 요약, 분석, 추천 정보를 종합하여 하나의 완성된 리뷰 리포트를 작성합니다. "
              "이 리포트는 유튜브 영상 제작자가 쉽게 활용할 수 있도록 가독성이 좋고, 핵심 포인트가 정리된 형태여야 합니다.",
    allow_delegation=False,
    verbose=True
)


# 🎯 {movie} 정보 조사 태스크
gather_movie_info = Task(
    description=(
        "입력받은 {movie}에 대한 상세 정보를 조사하고 정리합니다.\n"
        "1️⃣ {movie}의 개요 (감독, 출연진, 장르, 개봉일, 상영시간 등).\n"
        "2️⃣ 제작 과정 및 배경 스토리 조사.\n"
        "3️⃣ 개봉 당시의 평가 및 흥행 성적 분석.\n"
        "4️⃣ 리뷰어가 참고할 만한 흥미로운 정보 제공.\n"
    ),
    expected_output=(
        "1. {movie} 개요 및 주요 정보 (감독, 배우, 장르, 개봉일 등)\n"
        "2. 제작 배경 및 비하인드 스토리\n"
        "3. 평론가 및 관객 반응 정리\n"
        "4. 유튜버가 활용할 수 있는 흥미로운 정보\n"
    ),
    agent=movie_researcher,
)

# 🎯 {movie} 내용 요약 태스크
summarize_movie = Task(
    description=(
        "{movie}의 주요 내용을 요약하되, 스포일러를 최소화하여 전달합니다.\n"
        "1️⃣ {movie}의 기본적인 스토리라인 정리.\n"
        "2️⃣ 주요 사건 및 클라이맥스 전까지의 전개 설명.\n"
        "3️⃣ 시청자의 흥미를 유지하면서도 핵심 내용을 전달할 수 있도록 구성.\n"
    ),
    expected_output=(
        "1. {movie}의 전체적인 스토리라인 요약\n"
        "2. 주요 사건 및 전개 설명 (스포일러 최소화)\n"
        "3. 리뷰 영상에서 활용할 수 있는 핵심 스토리 정보\n"
    ),
    agent=movie_summarizer,
)

# 🎯 {movie} 해석 및 결말 분석 태스크
analyze_movie = Task(
    description=(
        "{movie}의 주제, 상징적 요소, 연출 의도 등을 분석하고 결말에 대한 해석을 제공합니다.\n"
        "1️⃣ {movie}의 주요 메시지와 테마 분석.\n"
        "2️⃣ 감독의 연출 의도 및 사용된 {movie}적 기법 분석.\n"
        "3️⃣ 결말에 대한 다양한 해석 및 시청자에게 생각할 거리를 제공.\n"
    ),
    expected_output=(
        "1. {movie}의 주제 및 주요 메시지 분석\n"
        "2. 연출 기법 및 상징 요소 정리\n"
        "3. 결말에 대한 다양한 해석 제공\n"
    ),
    agent=movie_analyst,
)

# 🎯 비슷한 {movie} 추천 태스크
recommend_movies = Task(
    description=(
        "분석한 {movie}와 유사한 스타일, 장르, 주제를 가진 {movie}를 추천합니다.\n"
        "1️⃣ 같은 장르의 {movie} 추천.\n"
        "2️⃣ 비슷한 연출 기법이나 분위기를 가진 {movie} 추천.\n"
        "3️⃣ 같은 감독이나 주요 배우가 출연한 {movie} 추천.\n"
        "4️⃣ 각 추천 {movie}에 대한 간략한 설명 및 추천 이유 제공.\n"
    ),
    expected_output=(
        "1. 유사한 장르의 {movie} 목록\n"
        "2. 비슷한 연출 기법을 활용한 {movie} 추천\n"
        "3. 같은 감독 또는 배우가 참여한 {movie} 리스트\n"
        "4. 각 추천 {movie}의 특징 및 추천 이유\n"
    ),
    agent=movie_recommender,
)

# 🎯 {movie} 리뷰 리포트 작성 태스크
generate_movie_report = Task(
    description=(
        "이전 태스크에서 수집한 정보를 바탕으로 {movie} 리뷰 리포트를 요약하지 않고 길게 작성합니다.\n"
        "1️⃣ 영화 개요 및 주요 정보 요약.\n"
        "2️⃣ 영화의 핵심적인 스토리 및 특징 정리.\n"
        "3️⃣ 영화의 해석 및 결말 분석 포함.\n"
        "4️⃣ 비슷한 영화 추천 목록 제공.\n"
        "5️⃣ 유튜브 영상 제작자가 활용하기 쉽게, 정보가 구조화된 형태로 리포트 작성.\n"
    ),
    expected_output=(
        "**🎬 {movie} 리뷰 리포트**\n\n"
        "📌 **영화 개요**\n"
        "- 제목: {movie}\n"
        "- 감독: ...\n"
        "- 출연진: ...\n"
        "- 장르: ...\n"
        "- 개봉일: ...\n\n"
        
        "📖 **스토리 요약**\n"
        "- (스포일러 없이 핵심만 정리)\n\n"
        
        "🔍 **해석 및 결말 분석**\n"
        "- (주제, 상징성, 감독 의도 등 분석)\n\n"
        
        "🎥 **비슷한 영화 추천**\n"
        "- (유사한 분위기의 영화 리스트 및 추천 이유)\n\n"
        
        "🎬 **유튜브 리뷰 포인트**\n"
        "- (영상에서 강조하면 좋을 포인트 정리)\n"
    ),
    agent=movie_reporter,
)

## 대본 버전

In [7]:
## 대본 버전

# 1️⃣ {movie} 정보 수집 에이전트
movie_researcher = Agent(
    role="{movie} 연구자",
    goal="입력받은 {movie}의 개요, 감독, 출연진, 장르, 제작 배경, 흥행 성적 등의 정보를 조사",
    backstory="당신은 {movie} 전문가로서, 특정 {movie}에 대한 자세한 정보를 수집하는 역할을 맡고 있습니다. "
              "{movie}의 제작 과정, 주요 배우 및 감독의 이력, 촬영 기법 등의 데이터를 신뢰할 수 있는 출처에서 조사합니다. "
              "또한, {movie}가 개봉했을 당시의 평가와 관객 반응도 함께 정리하여 유튜버가 리뷰에 활용할 수 있도록 제공합니다.",
    allow_delegation=False,
    verbose=True
)

# 2️⃣ {movie} 요약 에이전트
movie_summarizer = Agent(
    role="{movie} 요약가",
    goal="{movie}의 주요 내용을 스포일러를 최소화하면서 간결하고 효과적으로 요약",
    backstory="당신은 {movie} 리뷰 콘텐츠를 위한 전문 요약가입니다. "
              "{movie}의 주요 줄거리를 정리하되, 관객의 흥미를 유지할 수 있도록 핵심적인 요소를 강조하고 불필요한 세부사항은 배제합니다. "
              "{movie}의 전반적인 흐름을 전달하면서도, 스토리의 흥미를 유지할 수 있도록 스포일러 여부를 신중하게 고려합니다.",
    allow_delegation=False,
    verbose=True
)

# 3️⃣ {movie} 해석 및 결말 분석 에이전트
movie_analyst = Agent(
    role="{movie} 해석 전문가",
    goal="{movie}의 주제, 상징성, 감독의 연출 의도 등을 분석하고 결말에 대한 해석을 제공",
    backstory="당신은 {movie}의 메시지와 감독의 연출 의도를 심층적으로 분석하는 {movie} 평론가입니다. "
              "{movie}의 장면, 대사, 색채, 카메라 기법 등을 분석하여 {movie}가 전달하려는 의미를 파악합니다. "
              "또한, {movie}의 결말에 대한 다양한 해석을 제시하여 시청자가 더 깊이 이해할 수 있도록 도와줍니다.",
    allow_delegation=False,
    verbose=True
)

# 4️⃣ 비슷한 {movie} 추천 에이전트
movie_recommender = Agent(
    role="{movie} 추천 전문가",
    goal="분석한 {movie}와 유사한 스타일, 장르, 주제를 가진 {movie}를 추천",
    backstory="당신은 {movie} 추천 알고리즘 전문가로서, 특정 {movie}와 유사한 작품들을 분석하고 추천하는 역할을 맡고 있습니다. "
              "장르, 감독, 촬영 기법, 분위기, 스토리 구조 등을 고려하여 유사한 {movie}를 선정합니다. "
              "유튜버가 다양한 추천 리스트를 제공할 수 있도록, 각 추천 {movie}에 대한 간단한 설명과 추천 이유도 함께 작성합니다.",
    allow_delegation=False,
    verbose=True
)

# 5️⃣ {movie} 리뷰 리포트 작성 에이전트
movie_reporter = Agent(
    role="{movie} 영상 시나리오 작성자",
    goal="{movie}에 대한 종합적인 영화 유튜브 영상 시나리오를 작성",
    backstory="당신은 영화 추천 콘텐츠를 제작하는 크리에이터를 위한 전문 시나리오 작성자입니다. "
              "{movie}에 대한 조사, 요약, 분석, 추천 정보를 종합하여 하나의 완성된 유튜브 영상 시나리오 대본을 작성합니다. "
              "이 대본은 유튜브 영상 제작자가 바로 읽을 수 있도록 가독성이 좋고, 핵심 포인트가 정리된 형태여야 합니다.",
    allow_delegation=False,
    verbose=True
)


# 🎯 {movie} 정보 조사 태스크
gather_movie_info = Task(
    description=(
        "입력받은 {movie}에 대한 상세 정보를 조사하고 정리합니다.\n"
        "1️⃣ {movie}의 개요 (감독, 출연진, 장르, 개봉일, 상영시간 등).\n"
        "2️⃣ 제작 과정 및 배경 스토리 조사.\n"
        "3️⃣ 개봉 당시의 평가 및 흥행 성적 분석.\n"
        "4️⃣ 리뷰어가 참고할 만한 흥미로운 정보 제공.\n"
    ),
    expected_output=(
        "1. {movie} 개요 및 주요 정보 (감독, 배우, 장르, 개봉일 등)\n"
        "2. 제작 배경 및 비하인드 스토리\n"
        "3. 평론가 및 관객 반응 정리\n"
        "4. 유튜버가 활용할 수 있는 흥미로운 정보\n"
    ),
    agent=movie_researcher,
)

# 🎯 {movie} 내용 요약 태스크
summarize_movie = Task(
    description=(
        "{movie}의 주요 내용을 요약하되, 스포일러를 최소화하여 전달합니다.\n"
        "1️⃣ {movie}의 기본적인 스토리라인 정리.\n"
        "2️⃣ 주요 사건 및 클라이맥스 전까지의 전개 설명.\n"
        "3️⃣ 시청자의 흥미를 유지하면서도 핵심 내용을 전달할 수 있도록 구성.\n"
    ),
    expected_output=(
        "1. {movie}의 전체적인 스토리라인 요약\n"
        "2. 주요 사건 및 전개 설명 (스포일러 최소화)\n"
        "3. 리뷰 영상에서 활용할 수 있는 핵심 스토리 정보\n"
    ),
    agent=movie_summarizer,
)

# 🎯 {movie} 해석 및 결말 분석 태스크
analyze_movie = Task(
    description=(
        "{movie}의 주제, 상징적 요소, 연출 의도 등을 분석하고 결말에 대한 해석을 제공합니다.\n"
        "1️⃣ {movie}의 주요 메시지와 테마 분석.\n"
        "2️⃣ 감독의 연출 의도 및 사용된 {movie}적 기법 분석.\n"
        "3️⃣ 결말에 대한 다양한 해석 및 시청자에게 생각할 거리를 제공.\n"
    ),
    expected_output=(
        "1. {movie}의 주제 및 주요 메시지 분석\n"
        "2. 연출 기법 및 상징 요소 정리\n"
        "3. 결말에 대한 다양한 해석 제공\n"
    ),
    agent=movie_analyst,
)

# 🎯 비슷한 {movie} 추천 태스크
recommend_movies = Task(
    description=(
        "분석한 {movie}와 유사한 스타일, 장르, 주제를 가진 {movie}를 추천합니다.\n"
        "1️⃣ 같은 장르의 {movie} 추천.\n"
        "2️⃣ 비슷한 연출 기법이나 분위기를 가진 {movie} 추천.\n"
        "3️⃣ 같은 감독이나 주요 배우가 출연한 {movie} 추천.\n"
        "4️⃣ 각 추천 {movie}에 대한 간략한 설명 및 추천 이유 제공.\n"
    ),
    expected_output=(
        "1. 유사한 장르의 {movie} 목록\n"
        "2. 비슷한 연출 기법을 활용한 {movie} 추천\n"
        "3. 같은 감독 또는 배우가 참여한 {movie} 리스트\n"
        "4. 각 추천 {movie}의 특징 및 추천 이유\n"
    ),
    agent=movie_recommender,
)

# 🎯 {movie} 리뷰 리포트 작성 태스크
generate_movie_report = Task(
    description=(
        "이전 태스크에서 수집한 정보를 바탕으로 {movie} 유튜브 영상 대본을 요약하지 않고 길게 작성합니다.\n"
        "1️⃣ 영화 개요 및 주요 정보 요약.\n"
        "2️⃣ 영화의 핵심적인 스토리 및 특징 정리.\n"
        "3️⃣ 영화의 해석 및 결말 분석 포함.\n"
        "4️⃣ 비슷한 영화 추천 목록 제공.\n"
        "5️⃣ 유튜브 영상 제작자가 바로 읽으면 영상이 나오도록 대본을 자연스럽게 작성.\n"
    ),
    expected_output="각 섹션이 2~3개의 문단으로 구성된, "
        "잘 작성된 유튜브 대본."
        "실제로 읽기만 해도 영상 10분 분량이 나올법 한"
        "결과물은 한국어로 생성될 예정입니다.",
    agent=movie_reporter,
)

## 결과 출력

In [8]:
crew = Crew(
    agents=[movie_researcher, movie_summarizer, movie_analyst, movie_recommender, movie_reporter],
    tasks=[gather_movie_info, summarize_movie, analyze_movie, recommend_movies, generate_movie_report],
    verbose=2
)
result = crew.kickoff(inputs={"movie": "살인의 추억"})

 [DEBUG]: == Working Agent: 살인의 추억 연구자
 [INFO]: == Starting Task: 입력받은 살인의 추억에 대한 상세 정보를 조사하고 정리합니다.
1️⃣ 살인의 추억의 개요 (감독, 출연진, 장르, 개봉일, 상영시간 등).
2️⃣ 제작 과정 및 배경 스토리 조사.
3️⃣ 개봉 당시의 평가 및 흥행 성적 분석.
4️⃣ 리뷰어가 참고할 만한 흥미로운 정보 제공.



> Entering new CrewAgentExecutor chain...
Final Answer: 

1️⃣ **살인의 추억 개요 및 주요 정보**
   - **감독**: 봉준호
   - **출연진**: 
     - 송강호 (형사 박두만 역)
     - 김상경 (형사 이성규 역)
     - 전도연 (여자 피해자 역)
     - 이범수 (형사 강철구 역)
     - 기타: 이인희, 박철민, 원현준, 김병옥 등
   - **장르**: 범죄, 드라마, 스릴러
   - **개봉일**: 2003년 4월 24일
   - **상영시간**: 132분

2️⃣ **제작 과정 및 배경 스토리 조사**
   - "살인의 추억"은 1986년부터 1991년까지 실제로 발생한 화성 연쇄 살인 사건을 모티브로 한 영화로, 봉준호 감독이 각본과 연출을 맡았다. 이 영화는 한국 사회의 어두운 면모와 범죄 수사의 비극적인 현실을 조명한다. 제작 과정에서 봉준호 감독은 사건의 진실성을 유지하기 위해 많은 자료를 조사했으며, 실제 사건을 다룬 만큼 경찰의 수사 과정과 피해자 가족의 고통을 사실적으로 그려내기 위해 심혈을 기울였다. 또한, 1980년대의 한국 사회를 재현하기 위해 세트와 의상, 소품에도 많은 신경을 썼다.

3️⃣ **개봉 당시의 평가 및 흥행 성적 분석**
   - "살인의 추억"은 개봉 당시 비평가들로부터 극찬을 받았다. 봉준호 감독의 세밀한 연출과 배우들의 뛰어난 연기가 호평을 받으며, 한국 영화의 위상을 높이는 데 기여했다. 관객 반응 또한 매우 긍정적이었으며, 개봉 첫

In [9]:
from IPython.display import Markdown
Markdown(result)

안녕하세요, 여러분! 오늘은 영화 역사에 길이 남을 명작, 바로 봉준호 감독의 "살인의 추억"에 대해 이야기해보겠습니다. 이 영화는 2003년에 개봉하여, 한국 영화의 새로운 지평을 열었던 작품이죠. 자, 그럼 "살인의 추억"의 세계로 들어가 볼까요?

### 1️⃣ 영화 개요 및 주요 정보 요약

"살인의 추억"은 1986년부터 1991년까지 실제로 발생했던 화성 연쇄살인 사건을 바탕으로 한 범죄 드라마입니다. 봉준호 감독이 메가폰을 잡았고, 송강호, 김상경, 박해일 등 뛰어난 배우들이 출연했습니다. 이 영화는 대한민국의 사회적, 문화적 배경을 반영하며 범죄와 인간 심리를 깊이 있게 탐구합니다. 

이 영화는 단순한 범죄 수사극이 아닌, 당시 한국 사회의 억압적이고 부조리한 현실을 반영하고 있습니다. 영화의 분위기는 우울하고 음산하며, 관객에게 강한 긴장감을 주는 요소가 가득합니다. 특히, 봉준호 감독의 독특한 연출 스타일이 돋보이며, 각 인물의 심리와 갈등을 섬세하게 그려내어 관객을 매료시킵니다.

### 2️⃣ 영화의 핵심적인 스토리 및 특징 정리

영화는 화성에서 발생한 연쇄살인 사건을 중심으로 전개되며, 두 형사, 박두만(송강호)과 이형민(김상경)이 사건을 해결하기 위해 고군분투하는 모습을 담고 있습니다. 처음에는 단순한 성폭행 사건으로 시작되지만, 사건은 점점 심각해지고, 형사들은 범인을 잡기 위해 다양한 방법을 시도하게 됩니다. 

이 영화의 가장 큰 특징은 범죄 수사의 과정에서 드러나는 인간의 불안과 절망입니다. 범인을 찾기 위해 형사들은 고통받고, 그 과정에서 가족과 주변인들과의 관계도 위태로워집니다. 또한, 봉준호 감독은 사건을 해결하는 과정에서 보여지는 사회의 부조리와 실망을 강하게 드러내어, 관객에게 강렬한 메시지를 전달합니다.

### 3️⃣ 영화의 해석 및 결말 분석 포함

"살인의 추억"의 결말은 많은 사람들에게 충격과 여운을 남겼습니다. 사건의 범인은 끝내 잡히지 않고, 형사들은 결국 무력감을 느끼며 영화가 마무리됩니다. 이는 범죄 소설에서 흔히 볼 수 있는 해피엔딩과는 정반대의 결말로, 관객에게 불안감을 조성합니다. 

영화는 단순히 범죄 사건을 다루는 것이 아니라, 범죄가 일어나는 사회적 맥락과 그로 인해 영향을 받는 인물들의 심리를 깊이 있게 탐구합니다. 이는 우리 사회의 어두운 면을 드러내며, 관객에게 생각할 거리를 제공합니다. 결국, "살인의 추억"은 범죄를 다루면서도 인간의 본성과 사회의 부조리를 동시에 질문하는 작품입니다.

### 4️⃣ 비슷한 영화 추천 목록 제공

이제 "살인의 추억"을 감명 깊게 보신 분들을 위해 비슷한 주제와 스타일의 영화 몇 편을 추천해드리겠습니다.

- **"추격자" (2008)**: 연쇄살인범을 쫓는 형사의 이야기로, 범죄와 정의의 복잡성을 탐구합니다. 강렬한 긴장감이 특징입니다.
  
- **"악마를 보았다" (2010)**: 잔인한 범죄와 복수의 과정을 그린 작품으로, 인간 본성과 정의에 대한 질문을 던지는 영화입니다.

- **"버닝" (2018)**: 긴장감과 불안한 분위기를 조성하는 작품으로, 인물의 감정을 세밀하게 포착합니다.

- **"범죄와의 전쟁: 나쁜놈들 전성시대" (2012)**: 범죄 수사와 사회적 맥락을 동시에 조명하며, 유머와 드라마를 균형 있게 결합합니다.

- **"옥자" (2017)**: 봉준호 감독의 또 다른 작품으로, 사회적 비판과 감정적 스토리가 결합되어 있습니다.

- **"기생충" (2019)**: 사회적 계층과 범죄의 복잡한 관계를 탐구하며, 강렬한 스토리 전개가 돋보입니다.

이 추천 리스트를 통해 "살인의 추억"을 좋아하는 관객들이 유사한 주제와 스타일의 작품을 발견하고 즐길 수 있기를 바랍니다.

### 마무리

오늘 "살인의 추억"에 대한 이야기를 나누어 보았습니다. 이 영화는 단순한 범죄 수사를 넘어, 인간의 본성과 사회의 어두운 면을 깊이 있게 탐구하는 작품입니다. 많은 분들이 이 영화를 통해 새로운 시각을 갖게 되기를 바랍니다. 

이 영상이 도움이 되셨다면 좋아요와 구독 부탁드리며, 댓글로 여러분의 의견을 남겨주세요. 다음 영상에서 다시 만나요! 

감사합니다!